In [7]:
import subprocess
import time
import psutil  # For cross-platform memory usage monitoring
import os

class Testcase:
    def __init__(self, testcase_id, question_id, input_data, expected_output, time_limit, memory_limit, is_deleted):
        self.testcase_id = testcase_id
        self.question_id = question_id
        self.input_data = input_data
        self.expected_output = expected_output
        self.time_limit = time_limit
        self.memory_limit = memory_limit
        self.is_deleted = is_deleted

# Example user-submitted Python file path
user_code_file = "user_code.py"

# Test cases
testcases = [
    Testcase(1, 101, "1\n", "2\n", 2, 256 * 1024, False),  # Memory limit in KB
    Testcase(2, 101, "2\n", "4\n", 2, 256 * 1024, False),
    Testcase(3, 101, "3\n", "6\n", 2, 256 * 1024, False),
]

def run_user_code(file_path, input_data):
    """
    Run the user-submitted Python code file with the given input data and capture output.
    """
    try:
        process = psutil.Popen(
            [sys.executable, file_path],  # Command to run the user code file
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
        )
        # Start time
        start_cpu_time = time.process_time()
        
        # Provide input and capture output and errors
        output, error = process.communicate(input=input_data, timeout=testcase.time_limit)
        
        # End time
        cpu_time_used = time.process_time() - start_cpu_time
        
        # Check for runtime error
        if error:
            print(f"Runtime Error: {error.strip()}")
            print("EXIT: Runtime Error")
            process.kill()
            sys.exit(4)
        
        return output, cpu_time_used, process
    
    except subprocess.TimeoutExpired:
        print("Time Limit Exceeded")
        print("EXIT: Time Limit Exceeded")
        process.kill()
        sys.exit(2)

def check_memory_limit(process, memory_limit_kb):
    """
    Check if memory usage exceeds the limit.
    """
    memory_info = process.memory_info()
    memory_used_kb = memory_info.rss // 1024  # Convert bytes to KB
    return memory_used_kb > memory_limit_kb

def run_test_case(testcase):
    """
    Run a single test case.
    """
    if testcase.is_deleted:
        return  # Skip deleted test cases

    try:
        output, cpu_time_used, process = run_user_code(user_code_file, testcase.input_data)

        # Check for Time Limit Exceeded
        if cpu_time_used > testcase.time_limit:
            print(f"Test case {testcase.testcase_id} failed: Time Limit Exceeded")
            print("EXIT: Time Limit Exceeded")
            sys.exit(2)

        # Check for Memory Limit Exceeded
        if check_memory_limit(process, testcase.memory_limit):
            print(f"Test case {testcase.testcase_id} failed: Memory Limit Exceeded")
            print("EXIT: Memory Limit Exceeded")
            sys.exit(3)

        # Check for Wrong Answer
        if output.strip() != testcase.expected_output.strip():
            print(f"Test case {testcase.testcase_id} failed: Wrong Answer. Expected: '{testcase.expected_output}', Got: '{output}'")
            print("EXIT: Wrong Answer")
            sys.exit(1)

        print(f"Test case {testcase.testcase_id} passed in {cpu_time_used:.2f} seconds of CPU time.")

    except Exception as e:
        print(f"Test case {testcase.testcase_id} failed due to an exception: {str(e)}")
        print("EXIT: Runtime Error")
        sys.exit(4)

def run_all_tests():
    """
    Run all test cases and summarize results.
    """
    for testcase in testcases:
        run_test_case(testcase)

    print("All test cases passed.")
    print("EXIT: Pass All Testcases")
    sys.exit(0)

# Execute test cases
run_all_tests()


Test case 1 failed due to an exception: name 'testcase' is not defined
EXIT: Runtime Error


AttributeError: 'tuple' object has no attribute 'tb_frame'